# Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Implement

In [2]:
class Logistic_Regression:
    def __init__(self, alpha=0.0001, max_iter=1000, l2=0.01):
        ''' 
        Class constructor
        
        Parameters
        ----------
        alpha: the learning rate determines how big the step would be on each iteration.
        max_iter: number of times update weight
        l2: l2 regularization
        '''
        self.alpha = alpha
        self.max_iter = max_iter
        self.l2 = l2
        self.weight = None
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def fit(self, X, y):
        '''
        Trains Logistic Regression on the dataset (X, y).
        
        Parameters
        ----------
        X : numpy array, shape (m, n)
        The matrix of inputs
        y : numpy array, shape (m, 1) 
        The vector of outputs.
        '''
            
        # First column of this matrix is all ones (corresponding to x_0).
        X = np.append(np.ones((X.shape[0], 1)), X, axis=1)
        m, n = X.shape
        
        # Initialize weights shape (n + 1, 1)
        self.weight =  np.zeros((n, 1))
        
        for iter in range(self.max_iter):
            error = self.sigmoid(X @ self.weight) - y
            grad0 = X.T[0].dot(error) / m
            self.weight[0] -= self.alpha * grad0
            grad = (X.T[1:].dot(error) / m) + (self.l2 / m) * self.weight[1:]
            self.weight[1:] -= self.alpha * grad
            
            #Early stopping
            if np.linalg.norm(self.weight, 2) < 1e-6:
                break       
                
    def predict(self, X):
        '''
        Predict using the ridge model.
        
        Parameters
        ----------
        X : numpy array, shape (m, n)
        The matrix of inputs
        
        Return
        ----------
        Returns predicted values.
        '''
        # First column of this matrix is all ones (corresponding to x_0).
        X = np.append(np.ones((X.shape[0], 1)), X, axis=1)    
        return np.round(self.sigmoid(X @ self.weight)).astype(int)

In [3]:
def standardScaler(X):
    return (X - np.mean(X)) / np.std(X)

In [4]:
def r2(y_test, y_pred):
    base = np.sum((y_pred - y_test.mean()) ** 2)
    mse = np.sum((y_pred - y_test) ** 2)
    r2 = 1 - (mse / base)
    return r2

# Test

In [5]:
#Read data
data = pd.read_csv('Data/SAheart.csv')
data['famhist'] = data['famhist'].transform(lambda x : 1 if x=='Present' else 0)
data['chd'] = data['chd'].transform(lambda x : 1 if x=='Si' else 0)
data = data.to_numpy()

In [6]:
X = data[:,:9]
y = data[:,-1].reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Using l2 = 0.1
model = Logistic_Regression(alpha=0.1, max_iter=15000, l2=0.1)

X_train = standardScaler(X_train)
model.fit(X_train, y_train)

X_test = standardScaler(X_test)
y_pred = model.predict(X_test)

In [8]:
# Using l2 = 1
model = Logistic_Regression(alpha=0.4, max_iter=15000, l2=1)

model.fit(X_train, y_train)

y_pred1 = model.predict(X_test)

In [9]:
# Using l2 = 5
model = Logistic_Regression(alpha=0.4, max_iter=15000, l2=10)

model.fit(X_train, y_train)

y_pred2 = model.predict(X_test)

In [10]:
print('Score of using l2 = 0.1:', np.mean(y_pred == y_test))
print('Score of using l2 = 1:', np.mean(y_pred1 == y_test))
print('Score of using l2 = 5:', np.mean(y_pred2 == y_test))

Score of using l2 = 0.1: 0.7096774193548387
Score of using l2 = 1: 0.6666666666666666
Score of using l2 = 5: 0.6344086021505376
